In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
df = pd.read_csv('data/ecommerce_data.csv', encoding='latin1')

# Basic dataset information
print("Dataset Shape:", df.shape)
print("\nColumn Names:")
print(df.columns.tolist())
print("\nFirst 5 rows:")
print(df.head())
print("\nData Types:")
print(df.dtypes)
print("\nMissing Values:")
print(df.isnull().sum())

Dataset Shape: (541909, 8)

Column Names:
['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']

First 5 rows:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom  

Data Types:
InvoiceNo       object
StockCode    

In [2]:
# Create a copy for cleaning
df_clean = df.copy()

# Convert InvoiceDate to datetime
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])

# Create additional date columns for analysis
df_clean['Year'] = df_clean['InvoiceDate'].dt.year
df_clean['Month'] = df_clean['InvoiceDate'].dt.month
df_clean['Day'] = df_clean['InvoiceDate'].dt.day
df_clean['Weekday'] = df_clean['InvoiceDate'].dt.day_name()
df_clean['Hour'] = df_clean['InvoiceDate'].dt.hour

# Calculate total amount per transaction
df_clean['TotalAmount'] = df_clean['Quantity'] * df_clean['UnitPrice']

# Remove rows with negative quantities (returns) for sales analysis
df_sales = df_clean[df_clean['Quantity'] > 0].copy()

# Remove rows with zero or negative unit prices
df_sales = df_sales[df_sales['UnitPrice'] > 0].copy()

# Fill missing descriptions with 'Unknown Product'
df_sales['Description'] = df_sales['Description'].fillna('Unknown Product')

# Create customer segments based on available CustomerID
df_sales['CustomerType'] = df_sales['CustomerID'].apply(
    lambda x: 'Registered' if pd.notna(x) else 'Guest'
)

print("After cleaning:")
print(f"Original dataset: {df.shape[0]} rows")
print(f"Cleaned dataset: {df_sales.shape[0]} rows")
print(f"Removed {df.shape[0] - df_sales.shape[0]} rows")

# Check data quality
print(f"\nDate range: {df_sales['InvoiceDate'].min()} to {df_sales['InvoiceDate'].max()}")
print(f"Number of countries: {df_sales['Country'].nunique()}")
print(f"Number of unique products: {df_sales['StockCode'].nunique()}")
print(f"Total revenue: ${df_sales['TotalAmount'].sum():,.2f}")

After cleaning:
Original dataset: 541909 rows
Cleaned dataset: 530104 rows
Removed 11805 rows

Date range: 2010-12-01 08:26:00 to 2011-12-09 12:50:00
Number of countries: 38
Number of unique products: 3922
Total revenue: $10,666,684.54


In [3]:
# Save cleaned data
df_sales.to_csv('data/ecommerce_cleaned.csv', index=False)
print("Cleaned dataset saved to data/ecommerce_cleaned.csv")

Cleaned dataset saved to data/ecommerce_cleaned.csv


In [6]:
# Set plotting style
plt.style.use('seaborn-v0_8')

# 1. Revenue Analysis by Country
country_revenue = df_sales.groupby('Country')['TotalAmount'].sum().sort_values(ascending=False)
top_10_countries = country_revenue.head(10)

print("Top 10 Countries by Revenue:")
print(top_10_countries)

# 2. Monthly Revenue Trends
monthly_revenue = df_sales.groupby(['Year', 'Month'])['TotalAmount'].sum().reset_index()
monthly_revenue['YearMonth'] = monthly_revenue['Year'].astype(str) + '-' + monthly_revenue['Month'].astype(str).str.zfill(2)

print("\nMonthly Revenue Trends:")
print(monthly_revenue)

# 3. Top Products by Revenue
product_revenue = df_sales.groupby(['StockCode', 'Description'])['TotalAmount'].sum().sort_values(ascending=False)
top_20_products = product_revenue.head(20)

print("\nTop 20 Products by Revenue:")
print(top_20_products)

# 4. Customer Analysis
customer_analysis = df_sales[df_sales['CustomerType'] == 'Registered'].groupby('CustomerID').agg({
    'TotalAmount': 'sum',
    'InvoiceNo': 'nunique',
    'Quantity': 'sum'
}).rename(columns={'InvoiceNo': 'OrderCount'})

customer_analysis['AvgOrderValue'] = customer_analysis['TotalAmount'] / customer_analysis['OrderCount']

print(f"\nCustomer Insights:")
print(f"Total registered customers: {len(customer_analysis)}")
print(f"Average customer lifetime value: ${customer_analysis['TotalAmount'].mean():.2f}")
print(f"Average order value: ${customer_analysis['AvgOrderValue'].mean():.2f}")

# 5. Time-based patterns
hourly_sales = df_sales.groupby('Hour')['TotalAmount'].sum()
daily_sales = df_sales.groupby('Weekday')['TotalAmount'].sum()

print(f"\nPeak sales hour: {hourly_sales.idxmax()}:00 with ${hourly_sales.max():,.2f}")
print(f"Best performing day: {daily_sales.idxmax()} with ${daily_sales.max():,.2f}")

Top 10 Countries by Revenue:
Country
United Kingdom    9025222.084
Netherlands        285446.340
EIRE               283453.960
Germany            228867.140
France             209715.110
Australia          138521.310
Spain               61577.110
Switzerland         57089.900
Belgium             41196.340
Sweden              38378.330
Name: TotalAmount, dtype: float64

Monthly Revenue Trends:
    Year  Month  TotalAmount YearMonth
0   2010     12   823746.140   2010-12
1   2011      1   691364.560   2011-01
2   2011      2   523631.890   2011-02
3   2011      3   717639.360   2011-03
4   2011      4   537808.621   2011-04
5   2011      5   770536.020   2011-05
6   2011      6   761739.900   2011-06
7   2011      7   719221.191   2011-07
8   2011      8   759138.380   2011-08
9   2011      9  1058590.172   2011-09
10  2011     10  1154979.300   2011-10
11  2011     11  1509496.330   2011-11
12  2011     12   638792.680   2011-12

Top 20 Products by Revenue:
StockCode  Description       

In [5]:
# Save key metrics to files
top_10_countries.to_csv('output/top_countries_revenue.csv')
monthly_revenue.to_csv('output/monthly_revenue_trends.csv', index=False)
top_20_products.to_csv('output/top_products_revenue.csv')
customer_analysis.to_csv('output/customer_analysis.csv')

print("Analysis results saved to output folder")

Analysis results saved to output folder


In [7]:
import sqlite3

# Create SQLite database
conn = sqlite3.connect('data/ecommerce_analysis.db')

# Load cleaned data into SQL database
df_sales.to_sql('sales_transactions', conn, if_exists='replace', index=False)

# Create additional summary tables for analysis
# Country summary table
country_summary = df_sales.groupby('Country').agg({
    'TotalAmount': ['sum', 'mean', 'count'],
    'Quantity': 'sum',
    'CustomerID': lambda x: x.nunique()
}).round(2)
country_summary.columns = ['total_revenue', 'avg_order_value', 'total_orders', 'total_quantity', 'unique_customers']
country_summary = country_summary.reset_index()
country_summary.to_sql('country_performance', conn, if_exists='replace', index=False)

# Product summary table
product_summary = df_sales.groupby(['StockCode', 'Description']).agg({
    'TotalAmount': 'sum',
    'Quantity': 'sum',
    'InvoiceNo': 'nunique'
}).round(2)
product_summary.columns = ['total_revenue', 'total_quantity_sold', 'total_orders']
product_summary = product_summary.reset_index()
product_summary.to_sql('product_performance', conn, if_exists='replace', index=False)

print("Database created successfully with tables:")
print("1. sales_transactions - Main transaction data")
print("2. country_performance - Country-level metrics")
print("3. product_performance - Product-level metrics")
print("\nDatabase saved at: data/ecommerce_analysis.db")

conn.close()

Database created successfully with tables:
1. sales_transactions - Main transaction data
2. country_performance - Country-level metrics
3. product_performance - Product-level metrics

Database saved at: data/ecommerce_analysis.db


In [8]:
# 1. Main dashboard dataset with all necessary fields
tableau_main = df_sales[['InvoiceNo', 'StockCode', 'Description', 'Quantity', 
                        'InvoiceDate', 'UnitPrice', 'TotalAmount', 'CustomerID', 
                        'Country', 'Year', 'Month', 'Day', 'Weekday', 'Hour', 
                        'CustomerType']].copy()

# Add calculated fields for Tableau
tableau_main['Quarter'] = tableau_main['InvoiceDate'].dt.quarter
tableau_main['YearMonth'] = tableau_main['InvoiceDate'].dt.to_period('M').astype(str)
tableau_main['YearQuarter'] = tableau_main['Year'].astype(str) + '-Q' + tableau_main['Quarter'].astype(str)

# 2. Country performance summary for map visualization
country_tableau = df_sales.groupby('Country').agg({
    'TotalAmount': 'sum',
    'Quantity': 'sum',
    'InvoiceNo': 'nunique',
    'CustomerID': lambda x: x.nunique()
}).round(2)
country_tableau.columns = ['Total_Revenue', 'Total_Quantity', 'Total_Orders', 'Unique_Customers']
country_tableau['Avg_Order_Value'] = (country_tableau['Total_Revenue'] / country_tableau['Total_Orders']).round(2)
country_tableau['Revenue_Per_Customer'] = (country_tableau['Total_Revenue'] / country_tableau['Unique_Customers']).round(2)
country_tableau = country_tableau.reset_index()

# 3. Time series data for trend analysis
time_series = df_sales.groupby('InvoiceDate').agg({
    'TotalAmount': 'sum',
    'Quantity': 'sum',
    'InvoiceNo': 'nunique',
    'CustomerID': lambda x: x.nunique()
}).round(2)
time_series.columns = ['Daily_Revenue', 'Daily_Quantity', 'Daily_Orders', 'Daily_Customers']
time_series = time_series.reset_index()

# 4. Product category analysis (extract from description)
def categorize_product(description):
    description = str(description).upper()
    if any(word in description for word in ['HEART', 'LOVE', 'VALENTINE']):
        return 'Valentine/Romance'
    elif any(word in description for word in ['CHRISTMAS', 'XMAS', 'SANTA']):
        return 'Christmas'
    elif any(word in description for word in ['BAG', 'HANDBAG', 'TOTE']):
        return 'Bags'
    elif any(word in description for word in ['LIGHT', 'LAMP', 'LANTERN']):
        return 'Lighting'
    elif any(word in description for word in ['KITCHEN', 'COOKING', 'JAR', 'BOWL']):
        return 'Kitchen'
    elif any(word in description for word in ['DECORATION', 'ORNAMENT', 'DECOR']):
        return 'Decoration'
    else:
        return 'Other'

tableau_main['Product_Category'] = tableau_main['Description'].apply(categorize_product)

print("Tableau datasets created:")
print(f"Main dataset: {tableau_main.shape}")
print(f"Country summary: {country_tableau.shape}")
print(f"Time series: {time_series.shape}")
print(f"Product categories: {tableau_main['Product_Category'].value_counts()}")

Tableau datasets created:
Main dataset: (530104, 19)
Country summary: (38, 7)
Time series: (18499, 5)
Product categories: Product_Category
Other                354278
Valentine/Romance     57443
Bags                  50471
Christmas             24102
Lighting              18252
Kitchen               16741
Decoration             8817
Name: count, dtype: int64


In [9]:
# Export to CSV files for Tableau
tableau_main.to_csv('output/tableau_main_dataset.csv', index=False)
country_tableau.to_csv('output/tableau_country_summary.csv', index=False)
time_series.to_csv('output/tableau_time_series.csv', index=False)

# Create a data dictionary file
data_dictionary = """
# Tableau Data Dictionary - E-commerce Sales Analysis

## tableau_main_dataset.csv
- InvoiceNo: Unique invoice identifier
- StockCode: Product stock code
- Description: Product description
- Quantity: Number of items purchased
- InvoiceDate: Date and time of transaction
- UnitPrice: Price per unit
- TotalAmount: Total transaction amount
- Country: Customer country
- Year/Month/Day/Hour: Date components
- Weekday: Day of the week
- CustomerType: Registered or Guest customer
- Product_Category: Categorized product type
- Quarter/YearMonth/YearQuarter: Time groupings

## tableau_country_summary.csv
- Country: Country name
- Total_Revenue: Sum of all sales
- Total_Quantity: Total items sold
- Total_Orders: Number of unique orders
- Unique_Customers: Number of unique customers
- Avg_Order_Value: Average order amount
- Revenue_Per_Customer: Revenue divided by customers

## tableau_time_series.csv
- InvoiceDate: Date of transactions
- Daily_Revenue: Revenue per day
- Daily_Quantity: Items sold per day
- Daily_Orders: Orders per day
- Daily_Customers: Active customers per day
"""

with open('output/tableau_data_dictionary.txt', 'w') as f:
    f.write(data_dictionary)

print("\nAll datasets exported successfully:")
print("- tableau_main_dataset.csv")
print("- tableau_country_summary.csv") 
print("- tableau_time_series.csv")
print("- tableau_data_dictionary.txt")
print("\nReady for Tableau dashboard creation!")


All datasets exported successfully:
- tableau_main_dataset.csv
- tableau_country_summary.csv
- tableau_time_series.csv
- tableau_data_dictionary.txt

Ready for Tableau dashboard creation!
